In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import os
print(os.getcwd()) 

X = pd.read_csv('X_features.csv', index_col=0)
y = pd.read_csv('y_features.csv', index_col=0)
groups = X.pop('Feature_7')

test = pd.read_csv('test_features.csv', index_col=0).drop(columns = 'Feature_7')
sample = pd.read_csv('sample_submission_2.csv')
df_train = pd.read_csv('train.csv').set_index('Id')
df_test = pd.read_csv('test_2.csv').set_index('Id')
weights = df_train.loc[:,'Weight_Intraday':'Weight_Daily']

C:\Users\matze\Documents\Winton\CRISPDM


In [15]:
#Helper
def typecasting(df):
    categorial_cols = ['Feature_20','Feature_5']
    binary_cols = ['Feature_16']

    for col in categorial_cols:
        df[col] = df[col].astype('category')
    
    for col in binary_cols:
        df[col] = (df[col]-1).astype('category')

def trainscore(y_true,y_pred,weights):
    weights = weights.loc[y_true.index,:]
    daily = pd.concat([weights.Weight_Daily]*2,axis=1)
    minute = pd.concat([weights.Weight_Intraday]*60,axis=1)
    weights = pd.concat([minute,daily],axis=1)
    weights.columns = df_train.loc[:,'Ret_121':'Ret_PlusTwo'].columns
    
    
    wmea = (((abs(y_true-y_pred))*weights).values.flatten().sum()/y_true.size)
    
    
    #minute_ret = abs(y_true.loc[:,'Ret_121':'180']-y_pred.loc[:,'Ret_121':'Ret_180'])*weights.Weight_Intraday
    return wmea


def create_core_features(train,test):
    X_train=train.loc[:,'Feature_1':'Ret_MinusOne']
    X_val=test.loc[:,'Feature_1':'Ret_MinusOne']
    from sklearn.preprocessing import Imputer

    typecasting(X_train)
    X_train = pd.get_dummies(X_train,drop_first=True)

    typecasting(X_val)
    X_val = pd.get_dummies(X_val,drop_first=True)

    imputer = Imputer(strategy='mean', axis=0)
    X_train = pd.DataFrame(imputer.fit_transform(X_train), index= X_train.index,columns=X_train.columns)
    X_val = pd.DataFrame(imputer.fit_transform(X_val), index= X_val.index,columns=X_val.columns)
    X_train['Ret_MinutePast'] = train.loc[:,'Ret_2':'Ret_120'].sum(axis=1)
    X_val['Ret_MinutePast'] = test.loc[:,'Ret_2':'Ret_120'].sum(axis=1)
    return X_train, X_val

### Creating featuresets

In [17]:
core_features, core_features_test = create_core_features(df_train,df_test)
# generated features is X
#Adding the interactions to Featureset
crafted_features = pd.read_csv('BA_Features/X_features_BA.csv', index_col=0)
crafted_features_test = pd.read_csv('BA_Features/test_features_BA.csv', index_col=0)

In [21]:
crafted_features['interaction_3']= crafted_features.PR_120 * crafted_features.grouped_mad_fet_7
crafted_features_test['interaction_3']= crafted_features_test.PR_120 * test.grouped_mad_fet_7

#X['interaction_4']= X.smoothed_minute_mean * X.minute_mad
#test['interaction_4']= test.smoothed_minute_mean * test.minute_mad

crafted_features['interaction_5']= crafted_features.minute_mad * crafted_features.PR_120
crafted_features_test['interaction_5']= crafted_features_test.minute_mad * crafted_features_test.PR_120

crafted_features['interaction_6']= crafted_features.smoothed_minute_mean * crafted_features.grouped_mad_fet_5
crafted_features_test['interaction_6']= crafted_features_test.smoothed_minute_mean * crafted_features_test.grouped_mad_fet_5

crafted_features['interaction_7']= crafted_features.PR_120 * crafted_features.fet_7_Minute_MAD
crafted_features_test['interaction_7']= crafted_features_test.PR_120 * crafted_features_test.fet_7_Minute_MAD

crafted_features['interaction_8']= crafted_features.fet_7_Minute_MAD * crafted_features.fet_7_Minute_Mean
crafted_features_test['interaction_8']= crafted_features_test.fet_7_Minute_MAD * crafted_features_test.fet_7_Minute_Mean

print(crafted_features.shape,crafted_features_test.shape)

(40000, 102) (120000, 102)


In [293]:
all_features = pd.concat([core_features,crafted_features],axis=1).drop('Feature_7',axis='columns')
all_features_test = pd.concat([core_features_test,crafted_features_test],axis=1).drop('Feature_7',axis='columns')

features_and_target = all_features.copy()
features_and_target['Ret_PlusOne'] = y.Ret_PlusOne
features_and_target['Ret_PlusTwo'] = y.Ret_PlusTwo
features_and_target['Ret_MinutesFut'] = df_train.loc[:,'Ret_121':'Ret_180'].sum(axis=1)



In [294]:
import time
t0=time.time()
corr_spearman = features_and_target.corr(method='spearman').to_csv('BA_Features/spearman_cor.csv')
print("time:", round(time.time()-t0, 3), "s")
#corr_pearson = features_and_target.corr(method='pearson').to_csv('BA_Features/pearson_cor.csv')
#print("time:", round(time.time()-t0, 3), "s")

time: 111.148 s


In [ ]:
'''
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
n=20000

t0 = time.time()
mic_Ret_PlusOne = mutual_info_regression(all_features.sample(n,random_state=0), y.Ret_PlusOne.sample(n,random_state=0), random_state=0)
print('Time: ',time.time()-t0)
t0 = time.time()
#select10 = SelectKBest(score_func=mutual_info_regression, k=20).fit(all_features.sample(n,random_state=0), y.Ret_PlusOne.sample(n,random_state=0))
print('Time: ',time.time()-t0)
mic_Ret_PlusTwo = mutual_info_regression(all_features.sample(n,random_state=0), y.Ret_PlusTwo.sample(n,random_state=0), random_state=0)
print('Time: ',time.time()-t0)
t0 = time.time()
mic_Ret_Minute = mutual_info_regression(all_features.sample(n,random_state=0), df_train.loc[:,'Ret_121':'Ret_180'].sum(axis=1).sample(n,random_state=0), random_state=0)

print('Time: ',time.time()-t0)
'''

In [132]:
pd.DataFrame([mic_Ret_PlusOne,mic_Ret_PlusTwo,mic_Ret_Minute], index = ['MIC_PlusOne','MIC_PlusTwo','MIC_Minute'],columns=all_features.columns.values).T.to_csv('BA_Features/MIC.csv')

In [ ]:
'''from sklearn.preprocessing import PolynomialFeatures
poly=PolynomialFeatures()
X_poly = pd.read_csv('X_features_2.csv', index_col=0)
X_poly_test = pd.read_csv('test_features_2.csv', index_col=0)
X_poly = pd.DataFrame(poly.fit_transform(X_poly), columns= poly.get_feature_names(),index=X_poly.index).drop(['1'],axis='columns')
X_poly_test=pd.DataFrame(poly.fit_transform(X_poly_test), columns= poly.get_feature_names(),index=X_poly_test.index).drop(['1'],axis='columns')
display(X_poly.head())
display(X_poly_test.head())

X_poly_and_target = X_poly.copy()
X_poly_and_target['Ret_PlusOne'] = y.Ret_PlusOne
X_poly_and_target['Ret_PlusTwo'] = y.Ret_PlusTwo
X_poly_and_target['Ret_MinutesFut'] = df_train.loc[:,'Ret_121':'Ret_180'].sum(axis=1)

X_poly_and_target.corr(method='spearman').to_csv('BA_Features/poly_cor.csv')'''


In [133]:
MIC = pd.read_csv('BA_Features/MIC.csv', index_col=0)
corr_spearman = pd.read_csv('BA_Features/spearman_cor.csv',index_col=0).drop(['Ret_MinutesFut','Ret_PlusOne','Ret_PlusTwo'])
corr_poly = pd.read_csv('BA_Features/poly_cor.csv',index_col=0).drop(['Ret_MinutesFut','Ret_PlusOne','Ret_PlusTwo'])


In [481]:
MIC_poly = pd.read_csv('BA_Features/MIC_Poly.csv', index_col=0)

## Predicting Ret_PlusOne

### Featuresets (Needed once for each Target)

In [482]:
mic10 = MIC.MIC_PlusOne.sort_values(ascending=False)[0:10].index
mic20 = MIC.MIC_PlusOne.sort_values(ascending=False)[0:20].index
mic30 = MIC.MIC_PlusOne.sort_values(ascending=False)[0:30].index
corr10 = abs(corr_spearman.Ret_PlusOne).sort_values(ascending=False)[0:10].index
corr20 = abs(corr_spearman.Ret_PlusOne).sort_values(ascending=False)[0:20].index
corr30 = abs(corr_spearman.Ret_PlusOne).sort_values(ascending=False)[0:30].index
corrPear = abs(corr_pearson.Ret_PlusOne).sort_values(ascending=False)[0:20].index
core=core_features.drop('Feature_7',axis='columns').columns.values
poly15_1 = ['x1 x8','x3 x16', 'x5 x10', 'x5 x16', 'x5', 'x5 x6', 'x3 x6', 'x3 x5', 'x5 x11', 'x3 x10', 'x3^2', 'x2^2', 'x3', 'x3 x12']
poly15_2 = abs(corr_poly.Ret_PlusOne).sort_values(ascending=False)[0:15].index
micpoly15 = MIC_poly.MIC_PlusOne.sort_values(ascending=False)[0:15].index

### Ridge

In [483]:
#RFE Params
rfe_learner = Ridge()
rfe_params ={'alpha':100,'normalize':True,'max_iter':700}
rfe_learner = rfe_learner.set_params(**rfe_params)
rfe_target = 'Ret_PlusOne'

#RFE Fitting
rfe = RFE(rfe_learner,n_features_to_select=10, step=1, verbose=0)
rfe.fit(all_features,y[rfe_target])
print("training time:", round(time.time()-t0, 3), "s")
rfe10 = all_features.columns[rfe.get_support(indices=True)]

rfe = RFE(rfe_learner,n_features_to_select=20, step=1, verbose=0)
rfe.fit(all_features, y[rfe_target])
print("training time:", round(time.time()-t0, 3), "s")
rfe20 = all_features.columns[rfe.get_support(indices=True)]

rfe = RFE(rfe_learner,n_features_to_select=30, step=1, verbose=0)
rfe.fit(all_features,y[rfe_target])
print("training time:", round(time.time()-t0, 3), "s")
rfe30 = all_features.columns[rfe.get_support(indices=True)]

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupShuffleSplit
group_shuffle = GroupShuffleSplit(n_splits=10, test_size=0.60, train_size=0.4, random_state=0)
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.feature_selection import RFE

#Model_Params
train = pd.concat([all_features,X_poly],axis=1)
test = pd.concat([all_features_test,X_poly_test],axis=1)
learner = Ridge()
target= 'Ret_PlusOne'
param_grid= {'alpha': [20,19,18,17,16,15,12,11,10,8,6,4,2,1,0.5],'normalize':[True],'max_iter':[700]}

results = pd.DataFrame(columns=['algo','featureset','target','fitting_time','zerobenchmark','validationmark','mae','params'])


#Choose Featuresets
featuresets = pd.DataFrame([core,mic10,mic20,mic30,corr10,corr20,corr30,rfe10,rfe20,rfe30,poly15_1,poly15_2,micpoly15],index=['core','mic10','mic20','mic30','corr10','corr20','corr30','rfe10','rfe20','rfe30','polycorr15_1','polycorr15_2','micpoly15'],columns=list(range(1,43))).T
display(featuresets.columns)
#Fitting

for name in featuresets[['micpoly15','core']].columns:


    cols_to_use = featuresets[name].dropna()
    
    t0=time.time()
    gs= GridSearchCV(learner, param_grid, iid=False, refit=True, cv=group_shuffle, return_train_score =True, scoring='neg_mean_absolute_error')
    gs.fit(train[cols_to_use], y[target], groups)
    gs.cv_results_ 
    #display(gs.best_estimator_)
    #display(gs.best_score_ )
    display(name)
    fitting_time = time.time()-t0
    #Trainscore
    grid = pd.DataFrame(np.zeros([40000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns,index=df_train.index)
    grid.shape
    zerobenchmark= trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5)
    grid[target] = ((gs.predict(train[cols_to_use])))
    validationbenchmark = trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5) 


    ###Submission
    grid = pd.DataFrame(np.zeros([120000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns)
    grid.shape

    grid[target] = ((gs.predict(test[cols_to_use])))

    sample['Predicted'] = grid.values.flatten()
    sample.to_csv(('BA_Submissions/'+str(learner).split('(',1)[0]+'_'+name+target+".csv"), index=False)

    ###Output
    algo = str(learner).split('(',1)[0]
    params= gs.best_estimator_.get_params()
    mae = abs(gs.best_score_)
    results = results.append(pd.DataFrame([algo,name,target,fitting_time,zerobenchmark,validationbenchmark,mae,params],index=results.columns).T, ignore_index=True)


training time: 3308.568 s
training time: 3321.999 s
training time: 3335.429 s


Index(['core', 'mic10', 'mic20', 'mic30', 'corr10', 'corr20', 'corr30',
       'rfe10', 'rfe20', 'rfe30', 'polycorr15_1', 'polycorr15_2', 'micpoly15'],
      dtype='object')

'micpoly15'

'core'

In [382]:
from openpyxl import load_workbook
book = load_workbook('BA_Submissions/Results.xlsx')
writer = pd.ExcelWriter('BA_Submissions/Results.xlsx', engine='openpyxl')
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
writer.sheets['Tabelle1']
startrow = writer.sheets['Tabelle1'].max_row
results.to_excel(writer, index = False,header=False,startrow=startrow,sheet_name ='Tabelle1')
writer.save()

In [485]:
#rfe
from sklearn.ensemble import RandomForestRegressor
rfe_learner = RandomForestRegressor()
rfe_params= {'max_depth': 3,#tiefer
             'max_features': 'sqrt',#wahrscheinlich weniger glaub kürzere fitting time
             'min_samples_leaf': 1000, #höher
             'n_estimators': 200}#mehr bedeuten dauert länger
rfe_learner = rfe_learner.set_params(**rfe_params)
rfe_target = 'Ret_PlusOne'

#RFE Fitting
rfe = RFE(rfe_learner,n_features_to_select=10, step=1, verbose=1)
rfe.fit(all_features,y[rfe_target])
print("training time:", round(time.time()-t0, 3), "s")
#rfe10 = all_features.columns[rfe.get_support(indices=True)]


Fitting estimator with 144 features.
Fitting estimator with 143 features.
Fitting estimator with 142 features.
Fitting estimator with 141 features.
Fitting estimator with 140 features.
Fitting estimator with 139 features.
Fitting estimator with 138 features.
Fitting estimator with 137 features.
Fitting estimator with 136 features.
Fitting estimator with 135 features.
Fitting estimator with 134 features.
Fitting estimator with 133 features.
Fitting estimator with 132 features.


KeyboardInterrupt: 

### Huber

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupShuffleSplit
group_shuffle = GroupShuffleSplit(n_splits=10, test_size=0.60, train_size=0.4, random_state=0)
from sklearn.linear_model import HuberRegressor

from sklearn.feature_selection import RFE
#(estimator, n_features_to_select=None, step=1, verbose=0)

for name in featuresets.columns:
    cols_to_use = featuresets[name].dropna()

    train = all_features
    test = all_features_test
    #cols_to_use = ['Ret_MinusOne']
    learner = HuberRegressor()
    param_grid= {'alpha':[100,50,30,1,0.1],'epsilon':[1.35,1.8]}


    gs= GridSearchCV(learner, param_grid, iid=False, refit=True, cv=group_shuffle, return_train_score =True, scoring='neg_mean_absolute_error')
    gs.fit(train[cols_to_use], y.Ret_PlusOne, groups)
    gs.cv_results_ 
    display(gs.best_estimator_)
    display(gs.best_score_ )

    #Trainscore
    grid = pd.DataFrame(np.zeros([40000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns,index=df_train.index)
    grid.shape
    print("Zero Benchmark: ",trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5) )
    grid['Ret_PlusOne'] = ((gs.predict(train[cols_to_use])))
    print("Validation Benchmark: ",trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5) )


    ###Submission
    grid = pd.DataFrame(np.zeros([120000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns)
    grid.shape

    grid['Ret_PlusOne'] = ((gs.predict(test[cols_to_use])))

    sample['Predicted'] = grid.values.flatten()
    #sample.to_csv('submissions/featureset_RFE_20_2.csv', index=False)

### Random Forrest

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupShuffleSplit
group_shuffle = GroupShuffleSplit(n_splits=10, test_size=0.60, train_size=0.4, random_state=0)
from sklearn.ensemble import RandomForestRegressor

train = all_features
test = all_features_test
cols_to_use = ['Ret_MinusOne']



gs= GridSearchCV(learner, param_grid, iid=False, refit=True, cv=group_shuffle, return_train_score =True, scoring='neg_mean_absolute_error')
gs.fit(X[cols_to_use], y.Ret_PlusOne, groups)
gs.cv_results_ 
display(gs.best_estimator_)
display(gs.best_score_ )

#Trainscore
grid = pd.DataFrame(np.zeros([40000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns,index=df_train.index)
grid.shape
print("Zero Benchmark: ",trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5) )
grid['Ret_PlusOne'] = ((gs.predict(train[cols_to_use])))
print("Validation Benchmark: ",trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5) )


###Submission
grid = pd.DataFrame(np.zeros([120000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns)
grid.shape

grid['Ret_PlusOne'] = ((gs.predict(test[cols_to_use])))

sample['Predicted'] = grid.values.flatten()
#sample.to_csv('submissions/featureset_RFE_20_2.csv', index=False)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupShuffleSplit
group_shuffle = GroupShuffleSplit(n_splits=10, test_size=0.60, train_size=0.4, random_state=0)
from sklearn.ensemble import GradientBoostRegressor

train = all_features
test = all_features_test
cols_to_use = ['Ret_MinusOne']
learner = GradientBoostRegressor()
param_grid= {'bootstrap': [True, False],#kein plan also auch weg
             'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],#tiefer
             'max_features': ['auto', 'sqrt'],#wahrscheinlich weniger glaub kürzere fitting time
             'min_samples_leaf': [1, 2, 4], #höher
             'min_samples_split': [2, 5, 10],#kann wahrscheinlich weg
             'n_estimators': [200, 400, 600]}#mehr bedeuten dauert länger


gs= GridSearchCV(learner, param_grid, iid=False, refit=True, cv=group_shuffle, return_train_score =True, scoring='neg_mean_absolute_error')
gs.fit(X[cols_to_use], y.Ret_PlusOne, groups)
gs.cv_results_ 
display(gs.best_estimator_)
display(gs.best_score_ )

#Trainscore
grid = pd.DataFrame(np.zeros([40000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns,index=df_train.index)
grid.shape
print("Zero Benchmark: ",trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5) )
grid['Ret_PlusOne'] = ((gs.predict(train[cols_to_use])))
print("Validation Benchmark: ",trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5) )


###Submission
grid = pd.DataFrame(np.zeros([120000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns)
grid.shape

grid['Ret_PlusOne'] = ((gs.predict(test[cols_to_use])))

sample['Predicted'] = grid.values.flatten()
#sample.to_csv('submissions/featureset_RFE_20_2.csv', index=False)

# Ret_PlusTwo

In [472]:
mic10 = MIC.MIC_PlusTwo.sort_values(ascending=False)[0:10].index
mic20 = MIC.MIC_PlusTwo.sort_values(ascending=False)[0:20].index
mic30 = MIC.MIC_PlusTwo.sort_values(ascending=False)[0:30].index
corr10 = abs(corr_spearman.Ret_PlusTwo).sort_values(ascending=False)[0:10].index
corr20 = abs(corr_spearman.Ret_PlusTwo).sort_values(ascending=False)[0:20].index
corr30 = abs(corr_spearman.Ret_PlusTwo).sort_values(ascending=False)[0:30].index
#corrPear = abs(corr_pearson.Ret_PlusTwo).sort_values(ascending=False)[0:20].index
core=core_features.drop('Feature_7',axis='columns').columns.values
poly15_1 = ['x1 x8','x3 x16', 'x5 x10', 'x5 x16', 'x5', 'x5 x6', 'x3 x6', 'x3 x5', 'x5 x11', 'x3 x10', 'x3^2', 'x2^2', 'x3', 'x3 x12']
poly15_2 = poly15_2 = abs(corr_poly.Ret_PlusTwo).sort_values(ascending=False)[0:15].index
micpoly15 = MIC_poly.MIC_PlusOne.sort_values(ascending=False)[0:15].index

## Huber

In [484]:
#RFE Params
rfe_learner = HuberRegressor()
rfe_params ={'alpha':100,'max_iter':700}
rfe_learner = rfe_learner.set_params(**rfe_params)
rfe_target = 'Ret_PlusTwo'

#RFE Fitting
'''
rfe = RFE(rfe_learner,n_features_to_select=10, step=1, verbose=1)
rfe.fit(all_features,y[rfe_target])
print("training time:", round(time.time()-t0, 3), "s")
rfe10 = all_features.columns[rfe.get_support(indices=True)]

rfe = RFE(rfe_learner,n_features_to_select=20, step=1, verbose=0)
rfe.fit(all_features, y[rfe_target])
print("training time:", round(time.time()-t0, 3), "s")
rfe20 = all_features.columns[rfe.get_support(indices=True)]

rfe = RFE(rfe_learner,n_features_to_select=30, step=1, verbose=0)
rfe.fit(all_features,y[rfe_target])
print("training time:", round(time.time()-t0, 3), "s")
rfe30 = all_features.columns[rfe.get_support(indices=True)]
'''
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupShuffleSplit
group_shuffle = GroupShuffleSplit(n_splits=10, test_size=0.60, train_size=0.4, random_state=0)
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.feature_selection import RFE

#Model_Params
train = pd.concat([all_features,X_poly],axis=1)
test = pd.concat([all_features_test,X_poly_test],axis=1)
learner = HuberRegressor()
target= 'Ret_PlusTwo'
param_grid= {'alpha': [100,20,19,18,17,16,15,12,11],'epsilon':[1.35],'max_iter':[700]}

results = pd.DataFrame(columns=['algo','featureset','target','fitting_time','zerobenchmark','validationmark','mae','params'])


#Choose Featuresets
featuresets = pd.DataFrame([core,mic10,mic20,mic30,corr10,corr20,corr30,rfe10,rfe20,rfe30,poly15_1,poly15_2,micpoly15],index=['core','mic10','mic20','mic30','corr10','corr20','corr30','rfe10','rfe20','rfe30','polycorr15_1','polycorr15_2','micpoly15'],columns=list(range(1,43))).T
display(featuresets.columns)
#Fitting

for name in featuresets[['micpoly15','polycorr15_2']].columns:


    cols_to_use = featuresets[name].dropna()
    
    t0=time.time()
    gs= GridSearchCV(learner, param_grid, iid=False, refit=True, cv=group_shuffle, return_train_score =True, scoring='neg_mean_absolute_error')
    gs.fit(train[cols_to_use], y[target], groups)
    gs.cv_results_ 
    #display(gs.best_estimator_)
    #display(gs.best_score_ )
    display(name)
    fitting_time = time.time()-t0
    #Trainscore
    grid = pd.DataFrame(np.zeros([40000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns,index=df_train.index)
    grid.shape
    zerobenchmark= trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5)
    grid[target] = ((gs.predict(train[cols_to_use])))
    validationbenchmark = trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5) 


    ###Submission
    grid = pd.DataFrame(np.zeros([120000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns)
    grid.shape

    grid[target] = ((gs.predict(test[cols_to_use])))

    sample['Predicted'] = grid.values.flatten()
    sample.to_csv(('BA_Submissions/'+str(learner).split('(',1)[0]+'_'+name+target+".csv"), index=False)

    ###Output
    algo = str(learner).split('(',1)[0]
    params= gs.best_estimator_.get_params()
    mae = abs(gs.best_score_)
    results = results.append(pd.DataFrame([algo,name,target,fitting_time,zerobenchmark,validationbenchmark,mae,params],index=results.columns).T, ignore_index=True)


Index(['core', 'mic10', 'mic20', 'mic30', 'corr10', 'corr20', 'corr30',
       'rfe10', 'rfe20', 'rfe30', 'polycorr15_1', 'polycorr15_2', 'micpoly15'],
      dtype='object')

'micpoly15'

'polycorr15_2'

In [477]:
results

,algo,featureset,target,fitting_time,zerobenchmark,validationmark,mae,params
0,HuberRegressor,poly15_1,Ret_PlusTwo,31.4027,1773.92,1773.75,0.0151833,"{'alpha': 100, 'epsilon': 1.35, 'fit_intercept..."
1,HuberRegressor,poly15_2,Ret_PlusTwo,19.3254,1773.92,1773.26,0.0151685,"{'alpha': 11, 'epsilon': 1.35, 'fit_intercept'..."
